In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from utils import plot_subject_levels

In [2]:
df = pd.read_csv(r'/workspaces/miniconda/PKdata/data-raw/KI20160914/KI20160914.csv')

In [ ]:
plot_subject_levels(df)

In [4]:
from scipy.integrate import solve_ivp
import numpy as np
from tqdm import tqdm
from utils import one_compartment_model, objective_function

In [5]:
#prepare day 1 data
opt_df = df.dropna(subset = 'DV').copy()
opt_df['DV'] = opt_df['DV'].astype(pd.Float32Dtype())
opt_df = opt_df.loc[opt_df['DAY'] == 1, :]

#Within day 1 data, per subject identify the max concentration
#Drop time points occuring before the max, and set the time at that max conc to t=0
dfs = []
for c in opt_df['SUBJID'].drop_duplicates():
    work_df = opt_df.loc[opt_df['SUBJID'] == c, :].reset_index(drop = True)
    max_idx = work_df.loc[work_df['DV'] == work_df['DV'].max(), :].index[0]
    work_df = work_df.iloc[max_idx:, :]
    work_df['TIME'] = work_df['TIME'] - work_df['TIME'].min()
    dfs.append(work_df.copy())
work_df = pd.concat(dfs)

In [ ]:
#plot the prepared data
plot_subject_levels(work_df)

In [7]:
import numpy as np
from scipy.optimize import minimize
from joblib import dump, load
import os
from functools import partial
from utils import optimize_with_checkpoint_joblib

In [ ]:
from sklearn.preprocessing import RobustScaler
scale_df = work_df.copy()
#scale_df[['DV']] = RobustScaler().fit_transform(scale_df[['DV']])
mgkg_scaler = RobustScaler()
age_scaler = RobustScaler()
wt_scaler = RobustScaler()

scale_df['MGKG'] = (scale_df['DOSR'] / scale_df['WT'])
scale_df['WT_scale'] = wt_scaler.fit_transform(scale_df[['WT']])
scale_df['MGKG_scale'] = mgkg_scaler.fit_transform(scale_df[['MGKG']])
scale_df['AGE_scale'] = age_scaler.fit_transform(scale_df[['AGE']])
scale_df['DOSR'] = scale_df['DOSR'] / 100
plot_subject_levels(scale_df)

In [ ]:
scale_df

In [ ]:
from utils import OneCompartmentModel, ObjectiveFunctionColumn
mod_sse = OneCompartmentModel(dep_vars= {'k':[ ObjectiveFunctionColumn('AGE_scale'),
                                                ObjectiveFunctionColumn('SEX' )],
                                           'vd':[ObjectiveFunctionColumn('WT_scale',
                                                                         model_method='linear',
                                                                         
                                                                         allometric_norm_value=wt_scaler.transform([[70]])[0][0], 
                                                                         
                                                                         )]}, 
                              #loss_function=sum_of_squares_loss, 
                              optimizer_tol=None
                              )

In [11]:
init_summary = mod_sse.init_vals_pd

In [12]:
import pymc as pm

In [ ]:
model_params = init_summary.loc[init_summary['population_coeff'], :]
model_params

In [ ]:
model_param_dep_vars = init_summary.loc[init_summary['population_coeff'] == False, :]
model_param_dep_vars

In [ ]:
model_params.to_dict()

In [ ]:
model_param_dep_vars.to_dict()

In [17]:
def one_compartment_model(t, y, theta ):
    """
    Defines the differential equation for a one-compartment pharmacokinetic model.

    This function calculates the rate of change of drug concentration in the central 
    compartment over time.

    Args:
      t (float): Time point (not used in this specific model, but required by solve_ivp).
      y (list): Current drug concentration in the central compartment.
      k (float): Elimination rate constant.
      Vd (float): Volume of distribution.


    Returns:
      float: The rate of change of drug concentration (dC/dt).
    """
    k, Vd = theta
    C = y[0]  # Extract concentration from the state vector
    dCdt = -(k/Vd) * C  # Calculate the rate of change
    return dCdt

In [18]:
from pytensor.compile.ops import as_op
import pytensor.tensor as pt
import pytensor
from scipy.integrate import solve_ivp
import os

# Set PyTensor flags for debugging
os.environ["PYTENSOR_FLAGS"] = "optimizer=fast_compile,exception_verbosity=high"

In [19]:
debug_df = scale_df.loc[scale_df['SUBJID'].isin([1,2]), :]

In [20]:
all_subject_data = debug_df[['SUBJID', 'AGE_scale', 'WT_scale', 'SEX']].drop_duplicates(subset = 'SUBJID', keep='first').copy()

In [21]:
pm_df = debug_df.copy()
pm_subj_df = all_subject_data.copy()

In [22]:
coords = {'subject':pm_df['SUBJID'].unique()}

In [ ]:
all_subject_data

In [24]:
import logging

# Configure logging to capture console output
logging.basicConfig(filename='pymc_debug.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
old_subj_loop = True
pt_printing = True
with pm.Model(coords=coords) as model:
    
    
    betas = {}
    seen_coeff = []
    for idx, row in model_param_dep_vars.iterrows():
        coeff_name = row['model_coeff']
        
        beta_name = row['model_coeff_dep_var']
        if coeff_name not in seen_coeff:
            betas[coeff_name] = {}
        betas[coeff_name].update({beta_name:pm.Normal(f"beta_{coeff_name}_{beta_name}", mu = 0, sigma = 1)})
        seen_coeff.append(coeff_name)
        
    population_coeff = {}
    pop_coeff_intercept_mu = {}
    pop_coeff_intercept_sigma = {}
    pop_coeff_intercept_i = {}
    pm_model_params = []
    for idx, row in model_params.iterrows():
        coeff_name = row['model_coeff']
        population_coeff[coeff_name]=pm.Normal(f"{coeff_name}_pop", mu = 0, sigma = 1)
        pop_coeff_intercept_mu[coeff_name] = pm.Normal(f"{coeff_name}_intercept_mu", mu = 0, sigma = 1)
        pop_coeff_intercept_sigma[coeff_name] = pm.HalfNormal(f"{coeff_name}_intercept_sigma", sigma = 10)
        pop_coeff_intercept_i[coeff_name] = pm.Normal(f"{coeff_name}_intercept_sub",
                                                      mu = pop_coeff_intercept_mu[coeff_name], 
                                                      sigma = pop_coeff_intercept_sigma[coeff_name],
                                                      dims = 'subject'
                                                      )
        print(f"Shape of pop_coeff_intercept_i[{coeff_name}]: {pop_coeff_intercept_i[coeff_name].shape.eval()}")
        model_coeff = (population_coeff[coeff_name] + pop_coeff_intercept_i[coeff_name])
        for beta_name in betas[coeff_name]:
            print(f"Shape of model_coeff: {model_coeff.shape.eval()}")
            print(f"Shape of betas[{coeff_name}][{beta_name}]: {betas[coeff_name][beta_name].shape.eval()}")
            print(f"Shape of pm_subj_df[{beta_name}]: {pm_subj_df[beta_name].shape}")
            #print(f"Shape of pm_subj_df[{beta_name}][{sub_idx}]: {pm_subj_df[beta_name][sub_idx].shape}")
            model_coeff = pm.math.exp((model_coeff + betas[coeff_name][beta_name] * pm_subj_df[beta_name]))
        pm_model_params.append(
            pm.Deterministic(f"{coeff_name}_i", model_coeff, dims = 'subject')
        )
    all_conc = []  
    for sub_idx, subject in enumerate(pm_df['SUBJID'].unique()):
        subject_data = pm_df.loc[pm_df['SUBJID'] == subject, :]
        initial_conc = subject_data['DV'].values[0]#.item()
        t_eval = subject_data['TIME'].values
        t_span = [subject_data['TIME'].min(), subject_data['TIME'].max()]
        theta = [i[sub_idx] for i in pm_model_params]
        if old_subj_loop:
            @as_op(itypes=[pt.dscalar for i in pm_model_params], otypes=[pt.dvector])
            def pytensor_forward_model_matrix(*args):
                theta = [i for i in args]
                sol = solve_ivp(one_compartment_model, t_span, [initial_conc], t_eval=t_eval, args=(theta,) )
                ode_sol = sol.y[0]
                #print("\nShape of ode_sol within function:", ode_sol.shape)
                #print("\nValues of ode_sol within function:", ode_sol)
                return ode_sol
            
        
        #theta = pytensor.printing.Print("\nShape of theta before stack")(pt.shape(theta))
        #theta = pm.math.stack(theta)
        #theta = pytensor.printing.Print("\nShape of theta after stack")(pt.shape(theta))
        #print
        if old_subj_loop:
            ode_sol = pytensor_forward_model_matrix(*theta) #issue could be that this is not the same length for each subject
        else:
            sol = solve_ivp(one_compartment_model, t_span, [initial_conc], t_eval=t_eval, args=(*theta,) )
            #print(sol)
            ode_sol = sol.y[0] 
        if pt_printing:
            #_ = pytensor.printing.Print("Shape of ode_sol")(pt.shape(ode_sol))
            #ode_sol = pytensor.printing.Print("ode_sol Values:")(ode_sol)
        all_conc.append(ode_sol)
    all_conc = pt.concatenate(all_conc, axis=0)
    #if pt_printing:
    #    all_conc = pytensor.printing.Print("Shape of all_conc")(
    #    pt.shape(all_conc)
    #)
    sigma_obs = pm.HalfNormal("sigma_obs", sigma=1)
    pm.LogNormal("obs", mu=all_conc, sigma=sigma_obs, observed=pm_df["DV"].values.reshape(-1,1))
        
        

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
vars_list = list(model.values_to_rvs.keys())[:-1]

sampler = "DEMetropolisZ"
tune = draws = 2
with model:
    trace_DEMZ = pm.sample(step=[pm.DEMetropolisZ(vars_list)], tune=tune, draws=draws, chains = 1)
